<a href="https://colab.research.google.com/github/BossRobin/DeepLearningAlgorithmsByPytorch/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [0]:
train_dataset = torchvision.datasets.MNIST(root='./data', transform=transforms.ToTensor(), train=True, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
'''
nn.Conv2d(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True))
参数：
  in_channel:　输入数据的通道数，例RGB图片通道数为3；
  out_channel: 输出数据的通道数，这个根据模型调整；
  kennel_size: 卷积核大小，可以是int，或tuple；kennel_size=2,意味着卷积大小(2,2)， kennel_size=（2,3），意味着卷积大小（2，3）即非正方形卷积
  stride：步长，默认为1，与kennel_size类似，stride=2,意味着步长上下左右扫描皆为2， stride=（2,3），左右扫描步长为2，上下为3；
  padding：　零填充
BatchNorm2d()内部的参数如下：
  1.num_features：一般输入参数为batch_size*num_features*height*width，即为其中特征的数量
  2.eps：分母中添加的一个值，目的是为了计算的稳定性，默认为：1e-5
  3.momentum：一个用于运行过程中均值和方差的一个估计参数（我的理解是一个稳定系数，类似于SGD中的momentum的系数）
  4.affine：当设为true时，会给定可以学习的系数矩阵gamma和beta
class torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
参数：
  kernel_size(int or tuple) - max pooling的窗口大小，
  stride(int or tuple, optional) - max pooling的窗口移动的步长。默认值是kernel_size
  padding(int or tuple, optional) - 输入的每一条边补充0的层数
  dilation(int or tuple, optional) – 一个控制窗口中元素步幅的参数
  return_indices - 如果等于True，会返回输出最大值的序号，对于上采样操作会有帮助
  ceil_mode - 如果等于True，计算输出信号大小的时候，会使用向上取整，代替默认的向下取整的操作
'''
class ConvNet(nn.Module):
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    # Sequential是一个有序的容器，神经网络模块将按照在传入构造器的顺序依次被添加到计算图中执行，同时以神经网络模块为元素的有序字典也可以作为传入参数。
    self.layer1 = nn.Sequential( 
        # 二维卷积可以处理二维数据，
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2), 
        # 在卷积神经网络的卷积层之后总会添加BatchNorm2d进行数据的归一化处理，这使得数据在进行Relu之前不会因为数据过大而导致网络性能的不稳定
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(7*7*32, num_classes)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1) # 因为out的size是(100, 32, 7, 7),所以需要将out转换为(100, 7*7*32)以输入至全连接网络中
    out = self.fc(out)
    return out


In [0]:
model = ConvNet(num_classes).to(device)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images) # 和lstm不同，cnn直接输入图像即可，无须转换
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

In [0]:
'''
model的eval方法主要是针对某些在train和predict两个阶段会有不同参数的层。
比如Dropout层和BN层Dropout在train时随机选择神经元而predict要使用全部神
经元并且要乘一个补偿系数BN在train时每个batch做了不同的归一化因此也对应
了不同的参数，相应predict时实际用的参数是每个batch下参数的移动平均。
torch为了方便大家，设计这个eval方法就是让我们可以不用手动去针对这些层
做predict阶段的处理(也可以叫evaluation阶段,所以这个方法名才是eval)
这也就是说，如果模型中用了dropout或bn，那么predict时必须使用eval 
否则结果是没有参考价值的，不存在选择的余地。

作者：蔺笑天
链接：https://www.zhihu.com/question/363144860/answer/951669576
来源：知乎
著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。
'''
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')